In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

os.chdir("/content/drive/MyDrive/Credit Card Fraud Analysis")
print(os.getcwd())

/content/drive/MyDrive/Credit Card Fraud Analysis


In [3]:
!pip install -r requirements.txt

# Data reading and setting

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("credit_card_fraud_analysis") \
    .getOrCreate()

In [5]:
train_data = spark.read.csv("inputs/fraudTrain.csv", header=True, inferSchema=True)
test_data = spark.read.csv("inputs/fraudTest.csv", header=True, inferSchema=True)

In [6]:
train_data.show()

+---+---------------------+-------------------+--------------------+-------------+------+-----------+---------+------+--------------------+--------------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|_c0|trans_date_trans_time|             cc_num|            merchant|     category|   amt|      first|     last|gender|              street|                city|state|  zip|    lat|              long|city_pop|                 job|       dob|           trans_num| unix_time|         merch_lat|        merch_long|is_fraud|
+---+---------------------+-------------------+--------------------+-------------+------+-----------+---------+------+--------------------+--------------------+-----+-----+-------+------------------+--------+--------------------+----------+--------------------+----------+------------------+------------------+--------+
|  0|  2019-01-01 00:00:18|   2703186189

In [7]:
# Labeling the data and union
from pyspark.sql.functions import lit

train_set = train_data.withColumn("portion", lit('train'))
test_set = test_data.withColumn("portion", lit('test'))

full_data = train_set.union(test_set)
full_data.groupBy('portion').count().show()

+-------+-------+
|portion|  count|
+-------+-------+
|  train|1296675|
|   test| 555719|
+-------+-------+



In [8]:
del train_set, test_set

In [9]:
# Drop unnecessary columns

cols_to_drop = ['_c0', 'cc_num', 'first', 'last', 'street', 'zip', 'unix_time','trans_num']
full_data = full_data.drop(*cols_to_drop)

# Data processing and feature engineering

In [10]:
from pyspark.sql.functions import datediff, current_date, to_date, col

full_data = full_data.withColumn('age', (datediff(current_date(), to_date(col('dob'), 'yyyy-MM-dd')) / 365).cast('int')) \
                     .drop('dob')
full_data.show()

+---------------------+--------------------+-------------+------+------+--------------------+-----+-------+------------------+--------+--------------------+------------------+------------------+--------+-------+---+
|trans_date_trans_time|            merchant|     category|   amt|gender|                city|state|    lat|              long|city_pop|                 job|         merch_lat|        merch_long|is_fraud|portion|age|
+---------------------+--------------------+-------------+------+------+--------------------+-----+-------+------------------+--------+--------------------+------------------+------------------+--------+-------+---+
|  2019-01-01 00:00:18|fraud_Rippin, Kub...|     misc_net|  4.97|     F|      Moravian Falls|   NC|36.0788|          -81.1781|    3495|Psychologist, cou...|         36.011293|        -82.048315|       0|  train| 36|
|  2019-01-01 00:00:44|fraud_Heller, Gut...|  grocery_pos|107.23|     F|              Orient|   WA|48.8878|         -118.2105|     149|S

In [11]:
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, hour, dayofyear, to_timestamp
full_data = full_data.withColumn('trans_date_trans_time', to_timestamp('trans_date_trans_time', 'yyyy-MM-dd HH:mm:ss'))
full_data.printSchema()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- gender: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)
 |-- portion: string (nullable = false)
 |-- age: integer (nullable = true)



In [12]:
full_data.show()

+---------------------+--------------------+-------------+------+------+--------------------+-----+-------+------------------+--------+--------------------+------------------+------------------+--------+-------+---+
|trans_date_trans_time|            merchant|     category|   amt|gender|                city|state|    lat|              long|city_pop|                 job|         merch_lat|        merch_long|is_fraud|portion|age|
+---------------------+--------------------+-------------+------+------+--------------------+-----+-------+------------------+--------+--------------------+------------------+------------------+--------+-------+---+
|  2019-01-01 00:00:18|fraud_Rippin, Kub...|     misc_net|  4.97|     F|      Moravian Falls|   NC|36.0788|          -81.1781|    3495|Psychologist, cou...|         36.011293|        -82.048315|       0|  train| 36|
|  2019-01-01 00:00:44|fraud_Heller, Gut...|  grocery_pos|107.23|     F|              Orient|   WA|48.8878|         -118.2105|     149|S

In [13]:
full_data = full_data.withColumn('trans_month', month('trans_date_trans_time')) \
         .withColumn('trans_hour', hour('trans_date_trans_time')) \
         .withColumn('trans_day_of_week', dayofweek('trans_date_trans_time')) \
         .withColumn('trans_day_of_month', dayofmonth('trans_date_trans_time')) \
         .withColumn('trans_day_of_year', dayofyear('trans_date_trans_time')) \
         .drop('trans_date_trans_time')
full_data.show()

+--------------------+-------------+------+------+--------------------+-----+-------+------------------+--------+--------------------+------------------+------------------+--------+-------+---+-----------+----------+-----------------+------------------+-----------------+
|            merchant|     category|   amt|gender|                city|state|    lat|              long|city_pop|                 job|         merch_lat|        merch_long|is_fraud|portion|age|trans_month|trans_hour|trans_day_of_week|trans_day_of_month|trans_day_of_year|
+--------------------+-------------+------+------+--------------------+-----+-------+------------------+--------+--------------------+------------------+------------------+--------+-------+---+-----------+----------+-----------------+------------------+-----------------+
|fraud_Rippin, Kub...|     misc_net|  4.97|     F|      Moravian Falls|   NC|36.0788|          -81.1781|    3495|Psychologist, cou...|         36.011293|        -82.048315|       0|  t

## Calculating the Distance Between Merchant and Customer

In this section, we calculate the distance between two geographic points (latitude and longitude) using the **Haversine Formula**. This formula is commonly used to compute the **great-circle distance** between two points on the Earth's surface, providing an accurate measure of the distance between them.

### The Haversine Formula
The Haversine formula calculates the distance between two points on the Earth given their longitude and latitude in degrees. The formula is as follows:

$$
a = \sin^2\left(\frac{\Delta \phi}{2}\right) + \cos(\phi_1) \cdot \cos(\phi_2) \cdot \sin^2\left(\frac{\Delta \lambda}{2}\right)
$$

$$
c = 2 \cdot \text{atan2}\left(\sqrt{a}, \sqrt{1-a}\right)
$$

$$
d = R \cdot c
$$

Where:
- $$(\phi_1, \phi_2)$$ are the latitudes of the two points in radians.
- \(\lambda_1, \lambda_2\) are the longitudes of the two points in radians.
- \(\Delta \phi = \phi_2 - \phi_1\)
- \(\Delta \lambda = \lambda_2 - \lambda_1\)
- \(R\) is the radius of the Earth (mean radius = 6,371 km or 3,959 miles).
- \(d\) is the calculated distance between the two points.

### Steps in the Function

1. **Convert Degrees to Radians**: Latitude and longitude are provided in degrees, but trigonometric functions in the Haversine formula require the values to be in radians. The conversion from degrees to radians is done using the formula:
   $$ \text{radians} = \text{degrees} \times \left(\frac{\pi}{180}\right) $$

2. **Calculate the Differences in Coordinates**:
   - \(\Delta \phi = \phi_2 - \phi_1\) (difference in latitudes)
   - \(\Delta \lambda = \lambda_2 - \lambda_1\) (difference in longitudes)

3. **Apply the Haversine Formula**:
   The formula is used to compute the great-circle distance based on the spherical geometry of the Earth.

4. **Return the Distance**:
   The distance is computed in kilometers using the Earth's mean radius of 6,371 km. To compute the distance in miles, you can adjust the radius to 3,959 miles.

### Code Implementation in PySpark

We define a UDF (User Defined Function) to apply the Haversine formula across a PySpark DataFrame. The function takes the longitude and latitude values of two points and calculates the distance between them.



In [14]:
from pyspark.sql.functions import col, radians
from math import sin, cos, sqrt, atan2, radians

In [15]:
def haversine(lon1, lat1, lon2, lat2):
    # Radius of the Earth in kilometers
    R = 6371.0
    # Convert degrees to radians
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Difference in coordinates
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    # Haversine formula
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))

    # Calculate distance
    distance = R * c  # Distance in kilometers
    return distance

In [16]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType

haversine_udf = udf(haversine, FloatType())

full_data = full_data.withColumn("distance_km", haversine_udf(col("long"), col("lat"), col("merch_long"), col("merch_lat")))

# Saving

In [17]:
full_data.write.csv("inputs/processed_data", header=True)